In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import imghdr
import time
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

from sklearn.utils import shuffle

2024-11-01 21:08:11.403820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 21:08:11.418919: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 21:08:11.423517: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 21:08:11.435170: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-01 21:08:12.470319: W tensorflow/compiler/tf2

In [ ]:
import tensorflow as tf

# Check if TensorFlow can detect a GPU
if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
else:
    print("GPU is not available.")
    
# List all physical devices recognized by TensorFlow
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs detected:")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU detected.")

In [ ]:
# Define the brightness and contrast adjustment function
def adjust_brightness_contrast(image, brightness=40, contrast=1.0):
    # Convert to float to prevent clipping
    img = image.astype(np.float32)
    # Adjust brightness and contrast
    img = img * contrast + brightness
    # Clip to keep pixel values between 0 and 255 and convert back to uint8
    img = np.clip(img, 0, 255).astype(np.uint8)
    return img

# Ranges for brightness and contrast to test
brightness_values = range(-20, 71, 5)  # e.g., from -50 to 50 in steps of 20
contrast_values = [0.5, 0.75, 1.0, 1.25, 1.5]

In [ ]:
def is_image(file_path):
    # Returns the type of image if valid, otherwise None
    return imghdr.what(file_path) is not None

In [ ]:
# Initialize MediaPipe Hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Path where images are stored for each letter in the ASL alphabet
#data_dir = "../raw_data/asl_alphabet_dataset/asl_alphabet_train" # Change according to local dataset
#data_dir = "../raw_data/asl_cropped_dataset/asl_dataset"
data_dir = "../raw_data/ASL_Alphabet_Dataset/asl_alphabet_train"

landmark_data = []
labels = []

In [ ]:
# Collect landmarks for each letter
for letter in os.listdir(data_dir):
    #if letter=="C":
    #    break
    letter_dir = os.path.join(data_dir, letter)
    letter_count = 0
    for i, img_path in enumerate(os.listdir(letter_dir)):        
        if not is_image(os.path.join(letter_dir, img_path)):
            continue
        if i >= 300:
            break
        #print(os.path.join(letter_dir, img_path))
        img = cv2.imread(os.path.join(letter_dir, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            letter_count += 1
            landmarks = []
            for lm in results.multi_hand_landmarks[0].landmark:
                landmarks.extend([lm.x, lm.y, lm.z])  # Flattened landmark vector
            landmark_data.append(landmarks)
            labels.append(letter)  # Store the label (e.g., "A", "B", etc.)
    print(letter, letter_count)

In [ ]:
# Collect landmarks for each letter
for letter in os.listdir(data_dir):
    #if letter=="C":
    #    break
    letter_dir = os.path.join(data_dir, letter)
    letter_count = 0
    for i, img_path in enumerate(os.listdir(letter_dir)):        
        if not is_image(os.path.join(letter_dir, img_path)):
            continue
        #if i >= 300:
        #    break
        #print(os.path.join(letter_dir, img_path))
        img = cv2.imread(os.path.join(letter_dir, img_path))
        landmarks_found = False
        for brightness in brightness_values:
            for contrast in contrast_values:
                # Adjust brightness and contrast
                adjusted_image = adjust_brightness_contrast(img, brightness, contrast)                
                # Run MediaPipe hand detection
                results = hands.process(cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2RGB))
                # Check for hand landmarks and store them
                if results.multi_hand_landmarks:
                    landmarks = []
                    landmarks_found = True
                    letter_count += 1
                    for lm in results.multi_hand_landmarks[0].landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])  # Flattened landmark vector
                    landmark_data.append(landmarks)
                    labels.append(letter)  # Store the label (e.g., "A", "B", etc.)
                    break
            if landmarks_found:
                #print(f'Brightness: {brightness}, Contrast: {contrast}')
                break
    print(letter, letter_count)

In [ ]:
# Convert to arrays for model input
landmark_data = np.array(landmark_data)
labels = np.array(labels)

print(labels)
# Save the arrays to .npy files
np.save("landmark_data_short.npy", landmark_data)
np.save("labels_short.npy", labels)

# Normalize landmarks between 0 and 1
#landmark_data = landmark_data / np.max(landmark_data)

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

np.save('label_classes_short.npy', label_encoder.classes_)

In [2]:
labels = np.load("../models/production_model/labels_large.npy")
landmark_data = np.load("../models/production_model/landmark_data_large.npy")

# Normalize landmarks relative to the wrist (landmark 0) for each frame
normalized_landmark_data = []
for frame in landmark_data:
    # Extract wrist coordinates
    wrist_x, wrist_y, wrist_z = frame[0], frame[1], frame[2]

    # Normalize each landmark in the frame relative to the wrist
    normalized_frame = []
    for i in range(0, len(frame), 3):  # Iterate over (x, y, z) coordinates
        normalized_x = frame[i] - wrist_x
        normalized_y = frame[i + 1] - wrist_y
        normalized_z = frame[i + 2] - wrist_z
        normalized_frame.extend([normalized_x, normalized_y, normalized_z])

    normalized_landmark_data.append(normalized_frame)

# Convert to numpy array
normalized_landmark_data = np.array(normalized_landmark_data)

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [ ]:
landmark_data = np.array(landmark_data)
labels = np.array(labels)

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [ ]:
labels = np.load("../models/production_model/labels_large.npy")
landmark_data = np.load("../models/production_model/landmark_data_large.npy")

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

np.save('../models/production_model/label_classes_large.npy', label_encoder.classes_)

In [ ]:
labels = np.load("../models/production_model/labels_large.npy")
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

landmark_data = np.load("../models/production_model/landmark_data_large.npy")

normalized_landmark_data = []
def normalize_landmarks(landmarks):
    # Extract x and y coordinates
    x_coords = landmarks[::3]
    y_coords = landmarks[1::3]

    # Compute bounding box
    min_x, max_x = np.min(x_coords), np.max(x_coords)
    min_y, max_y = np.min(y_coords), np.max(y_coords)

    # Center the landmarks around (0, 0)
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    centered_landmarks = []
    for i in range(0, len(landmarks), 3):
        centered_landmarks.append(landmarks[i] - center_x)  # x - center_x
        centered_landmarks.append(landmarks[i + 1] - center_y)  # y - center_y
        centered_landmarks.append(landmarks[i + 2])  # z stays the same

    # Scale by diagonal of the bounding box to normalize for aspect ratio
    bbox_diagonal = np.sqrt((max_x - min_x) ** 2 + (max_y - min_y) ** 2)
    normalized_landmarks = [coord / bbox_diagonal for coord in centered_landmarks]

    return normalized_landmarks

# Normalize each frame individually
for frame in landmark_data:
    normalized_frame = normalize_landmarks(frame)
    normalized_landmark_data.append(normalized_frame)


#normalized_landmark_data = normalize_landmarks(landmark_data)
# Convert to numpy array
normalized_landmark_data = np.array(normalized_landmark_data)

In [3]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(normalized_landmark_data, labels_categorical, test_size=0.2, random_state=42)

In [4]:
# Shuffle training data and labels in unison
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Optionally shuffle test data too
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [5]:
# Build 1D CNN model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1730491730.093312  120509 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730491730.114676  120509 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730491730.114752  120509 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730491730.

In [ ]:
# Deeper CNN model with regularization
model = Sequential([
    # First convolutional block
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Second convolutional block
    Conv1D(128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Third convolutional block
    Conv1D(256, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Fully connected layers
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer for classification
])

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(patience = 5, restore_best_weights=True)

# Train the model
model.fit(X_train[..., np.newaxis], y_train, epochs=1000, batch_size=32, validation_data=(X_test[..., np.newaxis], y_test), callbacks=es)
#model.save("asl_sign_language_model.h5")
save_model(model, 'asl_sign_language_model_very_large_normalized_centered.keras')

Epoch 1/1000


I0000 00:00:1730491735.414925  120764 service.cc:146] XLA service 0x7fc9fc00a320 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730491735.414968  120764 service.cc:154]   StreamExecutor device (0): Quadro RTX 3000, Compute Capability 7.5
2024-11-01 21:08:55.446660: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-01 21:08:55.612728: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


  65/4640 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.0797 - loss: 3.2825 

I0000 00:00:1730491737.478093  120764 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4640/4640 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.7015 - loss: 1.0136 - val_accuracy: 0.9387 - val_loss: 0.2162
Epoch 2/1000
4640/4640 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9073 - loss: 0.3210 - val_accuracy: 0.9531 - val_loss: 0.1641
Epoch 3/1000
4640/4640 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9260 - loss: 0.2546 - val_accuracy: 0.9610 - val_loss: 0.1376
Epoch 4/1000
4640/4640 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9359 - loss: 0.2175 - val_accuracy: 0.9652 - val_loss: 0.1217
Epoch 5/1000
4640/4640 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9415 - loss: 0.1960 - val_accuracy: 0.9678 - val_loss: 0.1113
Epoch 6/1000
4640/4640 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9452 - loss: 0.1827 - val_accuracy: 0.9714 - val_loss: 0.1018
Epoch 7/1000
4640/4640 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.9505 - loss: 0.1649 - val_accuracy: 0.9723 - val_loss: 0.0965
Epoch 8/1000
4640/4640 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.9528 - loss

In [8]:
# Load the trained model
#model = tf.keras.models.load_model("asl_sign_language_model.h5")
model = tf.keras.models.load_model("asl_sign_language_model_very_large_wrist.keras")

ValueError: File not found: filepath=asl_sign_language_model_very_large_wrist.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
# Encode labels as integers and convert to categorical
labels = np.load("labels.npy")
landmark_data = np.load("landmark_data.npy")

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [7]:
# Initialize MediaPipe Hands and drawing utilities
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

scaling_factor = np.max(landmark_data)

# Initialize webcam
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("http://localhost:5000/video_feed")
cap = cv2.VideoCapture("http://192.168.2.31:5000/video_feed")

sequence = []
sequence_length = 1  # Set sequence length to 10 frames for rolling window approach - not working yet

while cap.isOpened():
    ret, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        landmarks = []
        #for lm in results.multi_hand_landmarks[0].landmark:
        #    landmarks.extend([lm.x, lm.y, lm.z])
        # Retrieve wrist coordinates (landmark 0)
        wrist = results.multi_hand_landmarks[0].landmark[0]

        for lm in results.multi_hand_landmarks[0].landmark:
            # Normalize each landmark relative to the wrist
            normalized_x = lm.x - wrist.x
            normalized_y = lm.y - wrist.y
            normalized_z = lm.z - wrist.z
            landmarks.extend([normalized_x, normalized_y, normalized_z])

        # Draw hand landmarks on the frame
        mp_drawing.draw_landmarks(
            frame, 
            results.multi_hand_landmarks[0], 
            mp_hands.HAND_CONNECTIONS
        )

        # Append new frame landmarks to sequence
        sequence.append(landmarks)
        if len(sequence) > sequence_length:
            sequence.pop(0)

        # Predict only if the sequence is full
        if len(sequence) == sequence_length:
            sequence_input = np.array(sequence).flatten()[np.newaxis, ..., np.newaxis]
            #sequence_input = sequence_input/scaling_factor
            prediction = model.predict(sequence_input)
            predicted_label_index = np.argmax(prediction)
            predicted_label = label_encoder.inverse_transform([predicted_label_index])
            confidence = prediction[0][predicted_label_index] * 100  # Get confidence percentage

            # Display prediction and confidence
            cv2.putText(frame, f"Predicted: {predicted_label[0]} ({confidence:.2f}%)", 
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)  # Black text color
    
    # Display the frame
    cv2.imshow("ASL Sign Language Detection", frame)
    
    # Press 'C' to terminate the loop
    if cv2.waitKey(1) & 0xFF == ord("c"):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1730492288.257482  120509 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1730492288.276502  125487 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: D3D12 (NVIDIA Quadro RTX 3000)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1730492288.293368  125461 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730492288.308763  125471 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730492307.729408  125468 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_d

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn

...

/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


/home/robert/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
import tensorflow as tf
import mediapipe as mp
import cv2
import numpy as np

# Load the trained model
model = tf.keras.models.load_model("asl_sign_language_model_large_normalized_centered.keras")

# Initialize MediaPipe Hands and drawing utilities
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Function to normalize landmarks relative to bounding box center and scale
def normalize_landmarks(landmarks):
    # Extract x and y coordinates
    x_coords = landmarks[::3]
    y_coords = landmarks[1::3]

    # Compute bounding box
    min_x, max_x = np.min(x_coords), np.max(x_coords)
    min_y, max_y = np.min(y_coords), np.max(y_coords)

    # Center the landmarks around (0, 0)
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    centered_landmarks = []
    for i in range(0, len(landmarks), 3):
        centered_landmarks.append(landmarks[i] - center_x)  # x - center_x
        centered_landmarks.append(landmarks[i + 1] - center_y)  # y - center_y
        centered_landmarks.append(landmarks[i + 2])  # z stays the same

    # Scale by diagonal of the bounding box to normalize for aspect ratio
    bbox_diagonal = np.sqrt((max_x - min_x) ** 2 + (max_y - min_y) ** 2)
    normalized_landmarks = [coord / bbox_diagonal for coord in centered_landmarks]

    return normalized_landmarks

# Initialize webcam
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("http://localhost:5000/video_feed")
cap = cv2.VideoCapture("http://192.168.2.31:5000/video_feed")

sequence = []
sequence_length = 1  # Set sequence length to 1 frame for testing

while cap.isOpened():
    ret, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        # Process landmarks for normalization
        landmarks = []
        for lm in results.multi_hand_landmarks[0].landmark:
            landmarks.extend([lm.x, lm.y, lm.z])

        # Normalize landmarks
        normalized_landmarks = normalize_landmarks(landmarks)

        # Draw hand landmarks on the frame
        mp_drawing.draw_landmarks(
            frame, 
            results.multi_hand_landmarks[0], 
            mp_hands.HAND_CONNECTIONS
        )

        # Append new frame landmarks to sequence
        sequence.append(normalized_landmarks)
        if len(sequence) > sequence_length:
            sequence.pop(0)

        # Predict only if the sequence is full
        if len(sequence) == sequence_length:
            sequence_input = np.array(sequence).flatten()[np.newaxis, ..., np.newaxis]
            prediction = model.predict(sequence_input)
            predicted_label_index = np.argmax(prediction)
            predicted_label = label_encoder.inverse_transform([predicted_label_index])
            confidence = prediction[0][predicted_label_index] * 100  # Get confidence percentage

            # Display prediction and confidence
            cv2.putText(frame, f"Predicted: {predicted_label[0]} ({confidence:.2f}%)", 
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)  # Black text color
    
    # Display the frame
    cv2.imshow("ASL Sign Language Detection", frame)
    
    # Press 'C' to terminate the loop
    if cv2.waitKey(1) & 0xFF == ord("c"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Load the trained model
model = tf.keras.models.load_model("asl_sign_language_model_large.keras")

labels = np.load("labels.npy")
#landmark_data = np.load("landmark_data.npy")

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Initialize MediaPipe Hands and drawing utilities
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Root directory with test images in subdirectories
root_test_dir = "../raw_data/test_set_pics"

# Initialize a list to store processed images
processed_images = []

# Iterate through each subdirectory in the root directory
for subdir_name in os.listdir(root_test_dir):
    subdir_path = os.path.join(root_test_dir, subdir_name)
    
    # Ensure it is a directory
    if os.path.isdir(subdir_path):
        actual_label = subdir_name  # Use the subdirectory name as the actual label

        # Iterate through each image in the subdirectory
        for img_name in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_name)
            print(img_path)
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            if results.multi_hand_landmarks:
                # Prepare landmarks for model prediction
                landmarks = []
                wrist = results.multi_hand_landmarks[0].landmark[0]
                for lm in results.multi_hand_landmarks[0].landmark:
                    # Normalize landmarks relative to the wrist
                    normalized_x = lm.x - wrist.x
                    normalized_y = lm.y - wrist.y
                    normalized_z = lm.z - wrist.z
                    landmarks.extend([normalized_x, normalized_y, normalized_z])

                # Prepare the input for the model
                sequence_input = np.array(landmarks)[np.newaxis, ..., np.newaxis]

                # Make prediction
                prediction = model.predict(sequence_input)
                predicted_label_index = np.argmax(prediction)
                predicted_label = label_encoder.inverse_transform([predicted_label_index])
                confidence = prediction[0][predicted_label_index] * 100  # Confidence percentage

                # Draw landmarks on the image
                mp_drawing.draw_landmarks(
                    img,
                    results.multi_hand_landmarks[0],
                    mp_hands.HAND_CONNECTIONS
                )

                # Add actual label, predicted label, and confidence to the image
                cv2.putText(img, f"Actual: {actual_label}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 255, 0), 6)  # Green text color
                cv2.putText(img, f"Predicted: {predicted_label[0]} ({confidence:.2f}%)", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 6)  # Red text color


            # Append processed image to the list
            processed_images.append(img)

# Display each processed image and check for "ESC" key press
for processed_img in processed_images:
    cv2.imshow("Processed Test Image", processed_img)
    key = cv2.waitKey(0)
    if key == 27:  # ESC key
        break

cv2.destroyAllWindows()


In [ ]:
import numpy as np
from scipy import ndimage

labels = np.load("labels.npy")
landmark_data = np.load("landmark_data.npy")

count_H = np.count_nonzero(labels == 'H')

print("Number of 'H's:", count_H)